# Download data

You can compose queries to download raw data.

In [ ]:
import doplaydo.dodata as dd
import matplotlib.pyplot as plt
import pandas as pd
import getpass

username = getpass.getuser()

PROJECT_ID = f"resistance-{username}"

You have access to:

- dd.Project
- dd.Die
- dd.Wafer
- dd.ParentCell
- dd.Cell
- dd.Device
- dd.DeviceData

Where each model has its table columns as attributes.


![](https://i.imgur.com/CfOHl57.png)

You can use `get_data_by_query` to query a subset of data filtered by a list of clauses.

It will return a list of tuples where the first element is a `DeviceData` object and the second one is a pandas DataFrame.


In [ ]:
data_tuples = dd.get_data_by_query([dd.Project.project_id == PROJECT_ID], limit=1)
device_data, df = data_tuples[0]  # each tuple has DeviceData and pd.Dataframe
device_data

In [ ]:
device_id = device_data.device.device_id

In [ ]:
df

In [ ]:
plt.plot(df["i"], df["v"])
plt.xlabel("I (A)")
plt.ylabel("V (V)")
plt.title(device_id)

You can aggregate the dataframes and the device data objects

In [ ]:
dfs = [dt[1] for dt in data_tuples]  # dataframes
dds = [dt[0] for dt in data_tuples]  # device data objects

You can use the `DeviceData` object to traverse the data model and access additional fields.

You can go from DeviceData to any other tables by following the dashed arrows.

Each column is an attribute on the object representing the table.

![](https://i.imgur.com/CfOHl57.png)

In [ ]:
print("device id: ", dds[0].device.device_id)
print("die x: ", dds[0].die.x)
print("die y: ", dds[0].die.y)
print("wafer id: ", dds[0].die.wafer.wafer_id)
print("cell id: ", dds[0].device.cell.cell_id)
print("parent cell id: ", dds[0].device.parent_cell.cell_id)
print("project id: ", dds[0].device.cell.project.project_id)

For example, you can reach the Cell table and all its columns:

In [ ]:
dds[0].device.cell

## Build table for JMP

We recommend doing all analysis in python but we also support exporting the data to a flat table for JMP or Excel.

In [ ]:
data_tuples = dd.get_data_by_query(
    [
        dd.Project.project_id == PROJECT_ID,
        dd.Device.device_id == device_id,
        dd.Die.x == 0,
        dd.Die.y == 0,
    ]
)
len(data_tuples)
dds = [dt[0] for dt in data_tuples]  # device data objects
dfs = [dt[1] for dt in data_tuples]  # dataframes
dfs[0]

In [ ]:
dfs_all = []

for device_data, df in zip(dds, dfs):
    df["device_id"] = device_data.device.device_id
    df["die_x"] = device_data.die.x
    df["die_y"] = device_data.die.y
    df["wafer_id"] = device_data.die.wafer.wafer_id
    df["cell_id"] = device_data.device.cell.cell_id
    df["parent_cell_id"] = device_data.device.parent_cell.cell_id
    dfs_all.append(df)

In [ ]:
dfs_all = pd.concat(dfs_all)  # You can concatenate all dataFrames together
dfs_all

## Advanced queries


To build advanced queries to filter metadata you can use the `attribute_filter` method. You can also use `or_`, `and_` for conditional clauses.

### Conditional filter

In [ ]:
data_tuples = dd.get_data_by_query(
    [dd.Project.project_id == PROJECT_ID, dd.Device.device_id == device_id]
)
len(data_tuples)

By default anything you pass to the list joins the clauses as `and_`

In [ ]:
data_tuples = dd.get_data_by_query(
    [dd.and_(dd.Project.project_id == PROJECT_ID, dd.Device.device_id == device_id)]
)
len(data_tuples)

You can also use an OR condition.

The query below filters by project id `PROJECT_ID` AND either  device  id (`rib_0p3` OR `rib_0p5`).

Because there are 21 measurements per device, one for each die, if you look for two specific devices, you will get 42 measurements.

In [ ]:
device_id1 = "resistance_resistance_sheet_W10_0_53000"
device_id2 = "resistance_resistance_sheet_W20_0_158000"

data_tuples = dd.get_data_by_query(
    [
        dd.Project.project_id == PROJECT_ID,
        dd.or_(dd.Device.device_id == device_id1, dd.Device.device_id == device_id2),
    ]
)
len(data_tuples)

You can also combine conditionals.

In the example below, you can get all the data for the specified project id, device id and either die coordinate.

In [ ]:
data_tuples = dd.get_data_by_query(
    [
        dd.Project.project_id == PROJECT_ID,
        dd.Device.device_id == device_id,
        dd.or_(
            dd.and_(dd.Die.x == 1, dd.Die.y == 1),  # either die 1,1
            dd.and_(dd.Die.x == 0, dd.Die.y == 0),  # or die 0,0
        ),
    ]
)
len(data_tuples)

### Attribute filter

You can filter attributes of any data models by passing a list of conditions as shown below.

You can use `dd.Cell/dd.Wafer/...`, `key`, `value`

You can only filter for values that are str, bool, int, float.

The query below filters Cells with waveguide width 0.3um that are in a particular project and die coordinates.

In [ ]:
device_data_objects = dd.get_data_by_query(
    [
        dd.attribute_filter(
            dd.Cell, "width_um", 100
        ),  # checking for values in JSON attributes
        dd.Project.project_id == PROJECT_ID,
        dd.Die.x == 0,
        dd.Die.y == 0,
    ]
)

In [ ]:
len(device_data_objects)

You can combine conditional and attribute filter clauses together.

You can get the data for Cells for a given project with length 100um and either width 20um or 100um

In [ ]:
data_tuples = dd.get_data_by_query(
    [
        dd.Project.project_id == PROJECT_ID,
        dd.Die.x == 0,
        dd.Die.y == 0,
        dd.attribute_filter(
            dd.Cell, "length_um", 20
        ),  # checking for values in JSON attributes
        dd.or_(
            dd.attribute_filter(dd.Cell, "width_um", 20),
            dd.attribute_filter(dd.Cell, "width_um", 100),
        ),
    ]
)
len(data_tuples)